In [1]:
import sklearn
import random 

# Pyspark Library #
# SQL
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import mean, col, split, regexp_extract, when, lit
# ML
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, IndexToString
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import os
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

In [2]:
#Make Spark Session
spark = SparkSession.builder.config( "spark.jars","/root/hadoop/share/hadoop/tools/lib/mysql-connector-java-8.0.27.jar") \
    .master("local").appName("ml_reco").getOrCreate()

In [70]:
#Load tables (buylist, product, category_small, user_recommand)
buylist = spark.read.format("jdbc").option("url","jdbc:mysql://b2cdb.chy6lfqzk3p1.ap-northeast-2.rds.amazonaws.com:3306/b2c")\
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "buylist") \
    .option("user", "admin").option("password", "SMoEcEXZ6PZUUiqDv5w9").load()
product = spark.read.format("jdbc").option("url","jdbc:mysql://b2cdb.chy6lfqzk3p1.ap-northeast-2.rds.amazonaws.com:3306/b2c")\
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "product") \
    .option("user", "admin").option("password", "SMoEcEXZ6PZUUiqDv5w9").load()
category_small = spark.read.format("jdbc").option("url","jdbc:mysql://b2cdb.chy6lfqzk3p1.ap-northeast-2.rds.amazonaws.com:3306/b2c")\
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "category_small") \
    .option("user", "admin").option("password", "SMoEcEXZ6PZUUiqDv5w9").load()
user_recommand = spark.read.format("jdbc").option("url","jdbc:mysql://b2cdb.chy6lfqzk3p1.ap-northeast-2.rds.amazonaws.com:3306/b2c")\
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "user_recommand") \
    .option("user", "admin").option("password", "SMoEcEXZ6PZUUiqDv5w9").load()
cart = spark.read.format("jdbc").option("url","jdbc:mysql://b2cdb.chy6lfqzk3p1.ap-northeast-2.rds.amazonaws.com:3306/b2c")\
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "cart") \
    .option("user", "admin").option("password", "SMoEcEXZ6PZUUiqDv5w9").load()
search_user = spark.read.format("jdbc").option("url","jdbc:mysql://b2cdb.chy6lfqzk3p1.ap-northeast-2.rds.amazonaws.com:3306/b2c")\
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "search_user") \
    .option("user", "admin").option("password", "SMoEcEXZ6PZUUiqDv5w9").load()
like = spark.read.format("jdbc").option("url","jdbc:mysql://b2cdb.chy6lfqzk3p1.ap-northeast-2.rds.amazonaws.com:3306/b2c")\
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "like_product") \
    .option("user", "admin").option("password", "SMoEcEXZ6PZUUiqDv5w9").load()

In [71]:
#필요한 값만 남기고 정리 (필요값 : user_id, product_id,
#                           category_(small,mid)_id, avg_star, keyword)
buylist = buylist.drop('buy_date','id','count')
product = product.drop('regist_time','name','price','brand','image')
category_small = category_small.drop('name')
search_user = search_user.drop('id')

In [74]:
product = product.withColumnRenamed('id','product_id')

In [75]:
product.show()

+----------+-----------------+--------+
|product_id|category_small_id|avg_star|
+----------+-----------------+--------+
|         1|                1|     4.5|
|         2|                1|     3.3|
|         3|                1|     2.8|
|         4|                3|     4.3|
|         5|                3|     4.2|
+----------+-----------------+--------+



In [79]:
df = cart.unionByName(like,allowMissingColumns=True)
df = df.unionByName(search_user,allowMissingColumns=True)
df = df.unionByName(buylist,allowMissingColumns=True)
df = pd.merge(df, product,on="product_id")

TypeError: Can only merge Series or DataFrame objects, a <class 'pyspark.sql.dataframe.DataFrame'> was passed

In [77]:
like.show()

+-------+----------+
|user_id|product_id|
+-------+----------+
|      1|         1|
+-------+----------+



In [78]:
df.show()

+-------+----------+-------+-----------------+--------+
|user_id|product_id|keyword|category_small_id|avg_star|
+-------+----------+-------+-----------------+--------+
|      1|         1|   null|             null|    null|
|      1|         1|   null|             null|    null|
|      1|         2|   null|             null|    null|
|      1|         3|   null|             null|    null|
|      1|         4|   null|             null|    null|
|      1|         5|   null|             null|    null|
|   null|         1|   null|                1|     4.5|
|   null|         2|   null|                1|     3.3|
|   null|         3|   null|                1|     2.8|
|   null|         4|   null|                3|     4.3|
|   null|         5|   null|                3|     4.2|
+-------+----------+-------+-----------------+--------+



+-------+----------+
|user_id|product_id|
+-------+----------+
|      1|         1|
|      1|         2|
|      1|         3|
|      1|         4|
|      1|         5|
+-------+----------+



+---+---------------+
| id|category_mid_id|
+---+---------------+
|  1|              1|
|  2|              1|
|  3|              1|
|  4|              1|
+---+---------------+

